In [1]:
import torch
import pandas as pd
import numpy as np
from torch.utils.data import Dataset
from torch.utils.tensorboard import SummaryWriter
import torch.nn as nn
writer = SummaryWriter()

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [29]:
df = pd.read_csv("/home/vaclavmatejka/devel/janestreet/model_8_SDAE_emphasized_loss/encoded.csv", nrows=324)
df.loc[400:,"enc_feature_0":"enc_feature_49"]
strip = 1 + df.shape[0] % 200
df.loc[:df.shape[0]-strip]

,Unnamed: 0,date,weight,resp_1,resp_2,resp_3,resp_4,resp,enc_feature_0,enc_feature_1,...,feature_120,feature_121,feature_122,feature_123,feature_124,feature_125,feature_126,feature_127,feature_128,feature_129
0,0,0.0,0.000000,0.009916,0.014079,0.008773,0.001390,0.006270,-1.154506,1.143241,...,0.000000,0.000000,1.168391,8.313583,1.782433,14.018213,2.653056,12.600292,2.301488,11.445807
1,1,0.0,16.673515,-0.002828,-0.003226,-0.007319,-0.011114,-0.009792,-1.750787,-0.484918,...,0.000000,0.000000,-1.178850,1.777472,-0.915458,2.831612,-1.417010,2.297459,-1.304614,1.898684
2,2,0.0,0.000000,0.025134,0.027607,0.033406,0.034380,0.023970,-1.676534,-0.037986,...,0.000000,0.000000,6.115747,9.667908,5.542871,11.671595,7.281757,10.060014,6.638248,9.427299
3,3,0.0,0.000000,-0.004730,-0.003273,-0.000461,-0.000476,-0.003200,-3.441493,-0.615681,...,0.000000,0.000000,2.838853,0.499251,3.033732,1.513488,4.397532,1.266037,3.856384,1.013469
4,4,0.0,0.138531,0.001252,0.002165,-0.001215,-0.006219,-0.002604,-0.633096,0.858466,...,0.000000,0.000000,0.344850,4.101145,0.614252,6.623456,0.800129,5.233243,0.362636,3.926633
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,195,0.0,0.000000,0.001245,0.003061,-0.001428,-0.004994,-0.000836,-0.857181,0.529766,...,-0.030492,4.015928,-0.616074,1.373682,-0.063830,3.991392,-0.380230,2.495774,-0.706637,1.326145
196,196,0.0,0.000000,0.001745,0.004459,0.002032,0.001878,0.005651,-0.173674,0.682367,...,0.000000,0.000000,0.045765,2.741755,0.402868,5.051385,0.562242,4.014664,0.119955,2.783974
197,197,0.0,1.861720,-0.000892,0.002704,0.001906,0.005911,0.005989,-2.054637,-0.022095,...,3.900376,4.012600,1.511398,0.223437,3.263071,3.642822,3.165782,1.600927,2.110370,0.620644
198,198,0.0,0.181539,-0.009904,-0.010052,-0.004704,0.017605,0.006678,0.701830,1.884682,...,0.000000,0.000000,4.179935,2.398382,2.975455,2.099968,4.687357,2.076800,5.053417,2.605069


In [3]:
t = torch.tensor([1.0,2000.0,3.0])
noise = torch.randn_like(t)
noise.multiply(1)

tensor([-0.1638,  0.1434,  0.7901])

In [4]:
VALIDATION_SIZE = 1000
df = pd.read_csv("/home/vaclavmatejka/devel/janestreet/jane-street-market-prediction/train.csv", nrows=10000)


In [5]:
df = df.fillna(0)
df.loc[df['resp'] <= 0, 'trade'] = 0
df.loc[df['resp'] > 0, 'trade'] = 1

In [6]:
df = df.sample(frac=1).reset_index()
df

,index,date,weight,resp_1,resp_2,resp_3,resp_4,resp,feature_0,feature_1,...,feature_122,feature_123,feature_124,feature_125,feature_126,feature_127,feature_128,feature_129,ts_id,trade
0,7573,1,0.000000,-0.004402,-0.008527,-0.025084,-0.026695,-0.019317,-1,-0.038037,...,-0.830452,-1.222913,-0.064535,0.873099,-0.033574,0.686286,-0.658388,-0.576456,7573,0.0
1,7931,1,0.980720,0.002422,-0.000294,0.003540,0.007488,0.002344,-1,0.451098,...,-1.878457,-1.679667,-1.111006,-1.041194,-1.845241,-0.843076,-1.612836,-0.763775,7931,1.0
2,2390,0,10.469229,-0.000664,-0.002001,-0.006666,-0.013447,-0.009634,1,-1.135190,...,-0.496450,-0.252105,-0.770537,-1.946120,-1.160092,-1.329745,-0.865972,-0.913463,2390,0.0
3,9875,1,0.327308,-0.006592,0.000331,0.007391,0.032523,0.027312,1,4.628062,...,4.258689,0.545369,2.595033,-0.850193,3.537147,-0.719482,3.897598,-0.088774,9875,1.0
4,3692,0,0.000000,0.000415,0.000405,0.002584,-0.000807,-0.005772,-1,-1.160377,...,-1.070417,-0.476946,-1.198019,-2.238900,-1.792717,-1.323538,-1.244437,-0.519089,3692,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,8882,1,0.290936,0.000292,0.002466,0.005289,0.019703,0.019392,1,-0.306043,...,4.214743,2.063822,3.263793,2.107301,4.698732,1.757803,4.615820,1.905804,8882,1.0
9996,7506,1,0.248309,-0.002285,-0.003088,-0.002118,-0.005039,-0.005047,-1,-3.172026,...,-0.165269,0.952018,-0.573792,-0.219193,-0.783141,-0.018819,-0.334602,0.636739,7506,0.0
9997,136,0,0.637961,0.005205,0.000224,0.002150,-0.013917,-0.016805,1,2.780710,...,4.642466,2.840470,4.100702,3.537450,4.956971,2.309757,4.489792,2.120206,136,0.0
9998,1141,0,0.524943,0.005839,0.012180,0.025380,-0.028579,-0.038777,1,-3.172026,...,1.881692,0.119740,2.341829,1.415651,2.555142,0.303994,1.680772,-0.358917,1141,0.0


In [7]:
class JaneStreetDataset(Dataset):

    # Constructor with defult values
    def __init__(self, df, transform=None):
        self.df = df
        self.len = len(self.df)
        self.transform = transform

    # Getter
    def __getitem__(self, index):
        sample = torch.from_numpy(self.df.iloc[index]["feature_0":"feature_129"].values), torch.tensor(self.df.iloc[index]["resp"])
        sample = sample[0].to(device), sample[1].to(device)
        if self.transform:
            sample = self.transform(sample)
        return sample

    # Get Length
    def __len__(self):
        return self.len

In [8]:
df.iloc[2]["resp"]

-0.009634158690232266

In [9]:
df.iloc[2]["feature_0":"feature_129"].values

array([ 1.00000000e+00, -1.13518959e+00, -1.61827057e+00,  4.54450598e-01,
        2.80194598e-01,  8.34325776e-01,  6.00991096e-01, -7.52922825e-01,
       -7.60245730e-01, -1.26065357e+00, -1.16282365e+00,  3.28691328e-01,
        1.57604453e-01, -2.00281266e+00, -1.54143486e+00, -8.06410352e-01,
       -1.44557732e+00,  8.54325656e-01,  8.53438605e-01,  2.70115762e-01,
        1.22521804e-01,  1.07888779e+00,  4.84688120e-01,  2.30981202e-01,
        9.86927946e-02,  2.75536780e-01,  1.03283244e-01, -5.49189303e-01,
       -7.48393220e-01, -5.29760682e-01, -4.92395888e-01, -6.75317837e-01,
       -1.07180458e+00, -1.06146682e+00, -9.24106051e-01, -7.42370466e-01,
       -4.48241035e-01,  1.07317754e+00,  9.00970932e-01, -2.40498460e-01,
       -3.47299546e-01,  2.17191018e+00, -3.95957967e-01,  1.14263502e+00,
        4.47306184e-01,  5.67436716e-01,  5.70503068e-01,  7.82952089e-01,
        4.61510203e-01,  1.99919843e-01,  1.22670170e+00,  2.65245924e+00,
        1.34119908e+00, -

In [10]:
DROPOUT_P = 0.0
big_number = 1000
small_number = 500
model = torch.nn.Sequential(
    torch.nn.Linear(130, small_number),
    nn.Dropout(p=DROPOUT_P),
    torch.nn.ReLU(),
    nn.BatchNorm1d(small_number),
    torch.nn.Linear(small_number, big_number),
    nn.Dropout(p=DROPOUT_P),
    torch.nn.ReLU(),
    nn.BatchNorm1d(big_number),
    torch.nn.Linear(big_number, big_number),
    nn.Dropout(p=DROPOUT_P),
    torch.nn.ReLU(),
    nn.BatchNorm1d(big_number),
    torch.nn.Linear(big_number, big_number),
    nn.Dropout(p=DROPOUT_P),
    torch.nn.ReLU(),
    nn.BatchNorm1d(big_number),
    torch.nn.Linear(big_number, small_number),
    nn.Dropout(p=DROPOUT_P),
    torch.nn.ReLU(),
    nn.BatchNorm1d(small_number),
    torch.nn.Linear(small_number, 1),
)
def init_weights(m):
    if type(m) == nn.Linear:
        torch.nn.init.xavier_uniform(m.weight)
        m.bias.data.fill_(0.01)

model.apply(init_weights)


<ipython-input-10-ae5219358239>:29: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  torch.nn.init.xavier_uniform(m.weight)


Sequential(
  (0): Linear(in_features=130, out_features=500, bias=True)
  (1): Dropout(p=0.0, inplace=False)
  (2): ReLU()
  (3): BatchNorm1d(500, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (4): Linear(in_features=500, out_features=1000, bias=True)
  (5): Dropout(p=0.0, inplace=False)
  (6): ReLU()
  (7): BatchNorm1d(1000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (8): Linear(in_features=1000, out_features=1000, bias=True)
  (9): Dropout(p=0.0, inplace=False)
  (10): ReLU()
  (11): BatchNorm1d(1000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (12): Linear(in_features=1000, out_features=1000, bias=True)
  (13): Dropout(p=0.0, inplace=False)
  (14): ReLU()
  (15): BatchNorm1d(1000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (16): Linear(in_features=1000, out_features=500, bias=True)
  (17): Dropout(p=0.0, inplace=False)
  (18): ReLU()
  (19): BatchNorm1d(500, eps=1e-05, momentum=0.1, affine=True, 

In [11]:
janestreet_train = JaneStreetDataset(df=df.iloc[:-VALIDATION_SIZE])
janestreet_validation = JaneStreetDataset(df=df.iloc[-VALIDATION_SIZE:])
BATCH_SIZE = 100
N_EPOCHS = 50
LR = 0.05
train_loader = torch.utils.data.DataLoader(dataset=janestreet_train, batch_size=BATCH_SIZE, drop_last=True)
validation_loader = torch.utils.data.DataLoader(dataset=janestreet_validation, batch_size=BATCH_SIZE, drop_last=True)

In [12]:
model = model.float()
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=LR)
# optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.001)
criterion = nn.MSELoss()

In [13]:
def train_model(model, train_loader, validation_loader, optimizer, criterion, n_epochs=None):
    accuracy_list = []
    loss_list = []
    for epoch in range(n_epochs):
        for x, y in train_loader:
            model.train()
            optimizer.zero_grad()
            z = model(x.float())
            y = y.view(-1, 1).float()
            loss = criterion(z, y)
            writer.add_scalar("Train data", loss, epoch)
            loss.backward()
            optimizer.step()
            loss_list.append(loss.data)
        # perform a prediction on the validation data
        for x_test, y_test in validation_loader:
            model.eval()
            z = model(x_test.float())
            y = y.view(-1, 1)
            loss = criterion(z, y.float())
            writer.add_scalar("Validation data", loss, epoch)
            accuracy_list.append(loss.data)
    return accuracy_list, loss_list

In [14]:
accuracy_list, loss_list = train_model(model, train_loader, validation_loader, optimizer, criterion, n_epochs=N_EPOCHS)

KeyboardInterrupt: 

In [ ]:
model.eval()

In [ ]:
index=2000
#a = torch.from_numpy(df.iloc[index]["feature_0":"feature_129"].values)
a = df.iloc[index]["feature_0":"feature_129"].values
resp = torch.tensor(df.iloc[index]["resp"])

r = []
for one in range(BATCH_SIZE):
    r.append(a)
r = np.array(r)
model_input =torch.from_numpy(r).float().to(device)
z = model(model_input)

(resp, z[0])

In [ ]:
accuracy_list

In [ ]:
loss_list